In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from optuna.samplers import TPESampler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:

# Загрузка данных
train = pd.read_excel('data/train.xlsx', index_col=0)
test = pd.read_excel('data/test.xlsx', index_col=0)

# Создание целевой переменной
train['Cancellation Status'] = train['Дата отмены'].apply(lambda x: 0 if pd.isna(x) else 1)

# Удаление столбцов, чтобы избежать утечки данных
train.drop(['Статус брони', 'Дата отмены'], axis=1, inplace=True)

# Объединение данных для консистентной предобработки
train_len = len(train)
data = pd.concat([train.drop('Cancellation Status', axis=1), test], axis=0)

In [3]:

# Функция для расширенного создания признаков
# Функция для расширенного создания признаков
def preprocess_data(df):
    # Конвертация дат в формат datetime
    df['Дата бронирования'] = pd.to_datetime(df['Дата бронирования'])
    df['Заезд'] = pd.to_datetime(df['Заезд'])
    df['Выезд'] = pd.to_datetime(df['Выезд'])
    
    # Создание новых признаков из дат
    df['booking_year'] = df['Дата бронирования'].dt.year
    df['booking_month'] = df['Дата бронирования'].dt.month
    df['booking_day'] = df['Дата бронирования'].dt.day
    df['booking_dayofweek'] = df['Дата бронирования'].dt.dayofweek
    df['booking_weekofyear'] = df['Дата бронирования'].dt.isocalendar().week
    df['booking_is_weekend'] = df['booking_dayofweek'].apply(lambda x: 1 if x >= 5 else 0)
    df['booking_hour'] = df['Дата бронирования'].dt.hour
    df['booking_minute'] = df['Дата бронирования'].dt.minute

    df['checkin_year'] = df['Заезд'].dt.year
    df['checkin_month'] = df['Заезд'].dt.month
    df['checkin_day'] = df['Заезд'].dt.day
    df['checkin_dayofweek'] = df['Заезд'].dt.dayofweek
    df['checkin_is_weekend'] = df['checkin_dayofweek'].apply(lambda x: 1 if x >= 5 else 0)

    df['checkout_year'] = df['Выезд'].dt.year
    df['checkout_month'] = df['Выезд'].dt.month
    df['checkout_day'] = df['Выезд'].dt.day
    df['checkout_dayofweek'] = df['Выезд'].dt.dayofweek
    df['checkout_is_weekend'] = df['checkout_dayofweek'].apply(lambda x: 1 if x >= 5 else 0)

    # Разница между датами
    df['booking_to_checkin_days'] = (df['Заезд'] - df['Дата бронирования']).dt.days
    df['checkin_to_checkout_days'] = (df['Выезд'] - df['Заезд']).dt.days
    df['booking_to_checkout_days'] = (df['Выезд'] - df['Дата бронирования']).dt.days

    # Создание признаков из 'Категория номера'
    df['Категория номера'] = df['Категория номера'].astype(str)
    df['room_category_length'] = df['Категория номера'].apply(len)
    df['room_category_words'] = df['Категория номера'].apply(lambda x: len(x.split()))
    df['room_category_unique_chars'] = df['Категория номера'].apply(lambda x: len(set(x)))
    df['room_category_digits'] = df['Категория номера'].apply(lambda x: sum(c.isdigit() for c in x))
    df['room_category_uppercase'] = df['Категория номера'].apply(lambda x: sum(c.isupper() for c in x))
    df['room_category_title_case'] = df['Категория номера'].apply(lambda x: sum(word.istitle() for word in x.split()))
    df['room_category_num'] = df['Категория номера'].str.extract(r'(\d+)').astype(float)  # Исправлено
    df['room_category_num'] = df['room_category_num'].fillna(df['room_category_num'].mean())

    # Дополнительные числовые признаки
    df['cost_per_night'] = df['Стоимость'] / df['Ночей']
    df['cost_per_room'] = df['Стоимость'] / df['Номеров']
    df['cost_per_guest'] = df['Стоимость'] / df['Гостей']
    df['rooms_per_guest'] = df['Номеров'] / df['Гостей']
    df['nights_per_guest'] = df['Ночей'] / df['Гостей']
    df['cost_per_night_per_room'] = df['Стоимость'] / (df['Ночей'] * df['Номеров'])
    df['cost_per_night_per_guest'] = df['Стоимость'] / (df['Ночей'] * df['Гостей'])
    df['prepayment_ratio'] = df['Внесена предоплата'] / df['Стоимость']
    df['remaining_payment'] = df['Стоимость'] - df['Внесена предоплата']
    df['remaining_payment_ratio'] = df['remaining_payment'] / df['Стоимость']

    # Флаги наличия предоплаты
    df['has_prepayment'] = df['Внесена предоплата'].apply(lambda x: 1 if x > 0 else 0)

    # Взаимодействия между признаками
    df['rooms_times_nights'] = df['Номеров'] * df['Ночей']
    df['guests_per_room'] = df['Гостей'] / df['Номеров']

    # Средняя стоимость на человека
    df['avg_cost_per_person'] = df['Стоимость'] / df['Гостей']

    # Дропаем исходные столбцы дат
    df.drop(['Дата бронирования', 'Заезд', 'Выезд', 'Категория номера'], axis=1, inplace=True)

    # Заполнение пропусков
    df['Внесена предоплата'] = df['Внесена предоплата'].fillna(0)
    df['room_category_num'] = df['room_category_num'].fillna(0)
    df['prepayment_ratio'] = df['prepayment_ratio'].fillna(0)
    df['remaining_payment_ratio'] = df['remaining_payment_ratio'].fillna(0)
    df['cost_per_night'] = df['cost_per_night'].replace([np.inf, -np.inf], np.nan).fillna(0)
    df['cost_per_room'] = df['cost_per_room'].replace([np.inf, -np.inf], np.nan).fillna(0)
    df['cost_per_guest'] = df['cost_per_guest'].replace([np.inf, -np.inf], np.nan).fillna(0)
    df['rooms_per_guest'] = df['rooms_per_guest'].replace([np.inf, -np.inf], np.nan).fillna(0)
    df['nights_per_guest'] = df['nights_per_guest'].replace([np.inf, -np.inf], np.nan).fillna(0)
    df['cost_per_night_per_room'] = df['cost_per_night_per_room'].replace([np.inf, -np.inf], np.nan).fillna(0)
    df['cost_per_night_per_guest'] = df['cost_per_night_per_guest'].replace([np.inf, -np.inf], np.nan).fillna(0)
    df['guests_per_room'] = df['guests_per_room'].replace([np.inf, -np.inf], np.nan).fillna(0)
    df['avg_cost_per_person'] = df['avg_cost_per_person'].replace([np.inf, -np.inf], np.nan).fillna(0)
    
    return df

In [4]:

data = preprocess_data(data)

In [5]:

# Удаление столбца '№ брони'
data = data.drop('№ брони', axis=1)

# Обработка категориальных признаков
categorical_cols = ['Способ оплаты', 'Источник', 'Гостиница']
data = pd.get_dummies(data, columns=categorical_cols)

# Очистка названий столбцов
data.columns = data.columns.str.replace(r'[^\w\s]', '', regex=True).str.replace(r'\s+', '_', regex=True)
data.columns = data.columns.astype(str)

In [6]:
# Разделение данных обратно на train и test
X = data.iloc[:train_len, :]
X_test = data.iloc[train_len:, :]
y = train['Cancellation Status']

# Стандартизация числовых признаков
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

# Отбор признаков с помощью модели LightGBM
lgb_clf = lgb.LGBMClassifier(n_estimators=1000, random_state=42)
lgb_clf.fit(X, y)

# Используем важность признаков для отбора
importances = pd.Series(lgb_clf.feature_importances_, index=X.columns)
importances = importances.sort_values(ascending=False)

[LightGBM] [Info] Number of positive: 5192, number of negative: 20982
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004428 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3819
[LightGBM] [Info] Number of data points in the train set: 26174, number of used features: 76
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.198365 -> initscore=-1.396546
[LightGBM] [Info] Start training from score -1.396546


In [7]:
# Выбираем топ-50 признаков
top_features = importances.head(50).index.tolist()
X_selected = X[top_features]
X_test_selected = X_test[top_features]

In [8]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

In [9]:
# Разделяем данные на обучающую и валидационную выборки для Optuna
X_train_optuna, X_valid_optuna, y_train_optuna, y_valid_optuna = train_test_split(
    X_selected, y, test_size=0.2, stratify=y, random_state=42
)

In [29]:
# Определение функции оптимизации для Optuna
def objective(trial):
    # Гиперпараметры для Random Forest
    rf_n_estimators = trial.suggest_int('rf_n_estimators', 100, 300)
    rf_max_depth = trial.suggest_int('rf_max_depth', 5, 20)
    rf_min_samples_split = trial.suggest_int('rf_min_samples_split', 2, 10)
    
    # Гиперпараметры для Gradient Boosting
    gb_n_estimators = trial.suggest_int('gb_n_estimators', 100, 350)
    gb_learning_rate = trial.suggest_loguniform('gb_learning_rate', 0.01, 0.3)
    gb_max_depth = trial.suggest_int('gb_max_depth', 3, 12)
    
    # Гиперпараметры для XGBoost
    xgb_n_estimators = trial.suggest_int('xgb_n_estimators', 100, 350)
    xgb_learning_rate = trial.suggest_loguniform('xgb_learning_rate', 0.01, 0.3)
    xgb_max_depth = trial.suggest_int('xgb_max_depth', 3, 12)
    
    # Гиперпараметры для CatBoost
    cat_iterations = trial.suggest_int('cat_iterations', 100, 350)
    cat_learning_rate = trial.suggest_loguniform('cat_learning_rate', 0.01, 0.3)
    cat_depth = trial.suggest_int('cat_depth', 3, 12)
    
    # Гиперпараметры для мета-модели (Logistic Regression)
    lr_C = trial.suggest_loguniform('lr_C', 1e-4, 10.0)
    
    # Определяем базовые модели с текущими гиперпараметрами
    base_learners = [
        ('rf', RandomForestClassifier(
            n_estimators=rf_n_estimators,
            max_depth=rf_max_depth,
            min_samples_split=rf_min_samples_split,
            random_state=42)),
        
        ('gb', GradientBoostingClassifier(
            n_estimators=gb_n_estimators,
            learning_rate=gb_learning_rate,
            max_depth=gb_max_depth,
            random_state=42)),
        
        ('xgb', XGBClassifier(
            n_estimators=xgb_n_estimators,
            learning_rate=xgb_learning_rate,
            max_depth=xgb_max_depth,
            use_label_encoder=False,
            eval_metric='logloss',
            random_state=42)),
        
        ('cat', CatBoostClassifier(
            iterations=cat_iterations,
            learning_rate=cat_learning_rate,
            depth=cat_depth,
            verbose=0,
            random_state=42))
    ]
    
    # Мета-модель
    meta_learner = LogisticRegression(
        C=lr_C,
        max_iter=1000,
        random_state=42)
    
    # Стэкинг-классификатор
    stacking_clf = StackingClassifier(
        estimators=base_learners,
        final_estimator=meta_learner,
        cv=5,
        n_jobs=-1
    )
    
    # Обучение модели на обучающей выборке
    stacking_clf.fit(X_train_optuna, y_train_optuna)
    
    # Предсказание на валидационной выборке
    y_pred_valid = stacking_clf.predict_proba(X_valid_optuna)[:, 1]
    
    # Оценка ROC AUC на валидационной выборке
    from sklearn.metrics import roc_auc_score
    roc_auc = roc_auc_score(y_valid_optuna, y_pred_valid)
    
    return -roc_auc  # Отрицательное значение для минимизации

In [ ]:

# Запуск исследования Optuna
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=150, n_jobs=-1)

# Вывод наилучших гиперпараметров
print("Наилучшие гиперпараметры:")
for key, value in study.best_params.items():
    print(f"{key}: {value}")

[I 2024-09-22 13:10:10,596] A new study created in memory with name: no-name-932a78f3-5956-4264-b17f-437c062d796c
[I 2024-09-22 13:13:49,982] Trial 1 finished with value: -0.8681226107463675 and parameters: {'rf_n_estimators': 272, 'rf_max_depth': 8, 'rf_min_samples_split': 9, 'gb_n_estimators': 201, 'gb_learning_rate': 0.28238434929594014, 'gb_max_depth': 5, 'xgb_n_estimators': 196, 'xgb_learning_rate': 0.034090268659426395, 'xgb_max_depth': 12, 'cat_iterations': 106, 'cat_learning_rate': 0.02637923510193758, 'cat_depth': 5, 'lr_C': 0.05194937873832614}. Best is trial 1 with value: -0.8681226107463675.
[I 2024-09-22 13:13:50,001] Trial 20 finished with value: -0.869252649956869 and parameters: {'rf_n_estimators': 133, 'rf_max_depth': 13, 'rf_min_samples_split': 7, 'gb_n_estimators': 311, 'gb_learning_rate': 0.010460503737452524, 'gb_max_depth': 5, 'xgb_n_estimators': 249, 'xgb_learning_rate': 0.05134833293197279, 'xgb_max_depth': 9, 'cat_iterations': 162, 'cat_learning_rate': 0.183950

In [22]:
# Вывод наилучших гиперпараметров
print("Наилучшие гиперпараметры:")
for key, value in study.best_params.items():
    print(f"{key}: {value}")


Наилучшие гиперпараметры:
rf_n_estimators: 258
rf_max_depth: 19
rf_min_samples_split: 4
gb_n_estimators: 197
gb_learning_rate: 0.04138497600216118
gb_max_depth: 3
xgb_n_estimators: 179
xgb_learning_rate: 0.07464105371778802
xgb_max_depth: 4
cat_iterations: 220
cat_learning_rate: 0.03332449131296827
cat_depth: 8
lr_C: 0.8502376470004751


In [10]:
import optuna.visualization as vis

vis.plot_optimization_history(study)
vis.plot_param_importances(study)

NameError: name 'study' is not defined

In [11]:
# Обновляем модели с лучшими гиперпараметрами
#best_params = study.best_params

base_learners = [
    ('rf', RandomForestClassifier(
        n_estimators=177,
        max_depth=5,
        min_samples_split=10,
        random_state=42)),
    
    ('gb', GradientBoostingClassifier(
        n_estimators=249,
        learning_rate=0.014558979708316613,
        max_depth=9,
        random_state=42)),
    
    ('xgb', XGBClassifier(
        n_estimators=349,
        learning_rate=0.010495089275623674,
        max_depth=3,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42)),
    
    ('cat', CatBoostClassifier(
        iterations=347,
        learning_rate=0.03505955989906093,
        depth=12,
        verbose=0,
        random_state=42))
]

meta_learner = LogisticRegression(
    C=3.9214510817910027,
    max_iter=1000,
    random_state=42)

stacking_clf = StackingClassifier(
    estimators=base_learners,
    final_estimator=meta_learner,
    cv=5,
    n_jobs=-1
)

In [12]:
# Кросс-валидация на всей обучающей выборке с найденными гиперпараметрами
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(stacking_clf, X_selected, y, cv=cv, scoring='roc_auc', n_jobs=-1)
print(f'\nROC AUC Scores from Cross-Validation: {scores}')
print(f'Mean ROC AUC Score from Cross-Validation: {scores.mean()}')

KeyboardInterrupt: 

In [13]:

# Обучение модели на всей обучающей выборке
stacking_clf.fit(X_selected, y)

StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(max_depth=5,
                                                       min_samples_split=10,
                                                       n_estimators=177,
                                                       random_state=42)),
                               ('gb',
                                GradientBoostingClassifier(learning_rate=0.014558979708316613,
                                                           max_depth=9,
                                                           n_estimators=249,
                                                           random_state=42)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_byno...
                                              max_delta_step=None, max_depth=3,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=349, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=42, ...)),
                               ('cat',
                                <catboost.core.CatBoostClassifier object at 0x000002DB02782B70>)],
                   final_estimator=LogisticRegression(C=3.9214510817910027,
                                                      max_iter=1000,
                                                      random_state=42),
                   n_jobs=-1)

In [14]:

# Предсказания на тестовых данных
test_predictions = stacking_clf.predict_proba(X_test_selected)[:, 1]

In [15]:
# Создание файла с предсказаниями
submission = pd.DataFrame({
    'Cancellation Probability': test_predictions
})

submission['Cancellation Probability'] = submission['Cancellation Probability'].round(7)

# Сохранение файла
submission.to_csv('submission_final.csv', index=False, header=False)